# NDP EP Tutorial: Dogs Service API Management

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sci-ndp/pop/blob/main/docs/dogs_service_api_tutorial.ipynb)
[![Open in Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/sci-ndp/pop/main?filepath=docs/dogs_service_api_tutorial.ipynb)

> 🚀 **Run Online Options:**
> - **Google Colab**: Dependencies installed automatically in the first cell
> - **Binder**: Pre-configured environment, ready to run immediately
> - **Local**: Requires `pip install requests jupyter`

This notebook demonstrates how to use the NDP EP API to create and manage external service integrations using the Dogs API as an example. You will learn how to:

1. **Create a service** that connects to an external API (Dog CEO API)
2. **Use redirect endpoints** to proxy requests to the external service
3. **Test service functionality** with various API endpoints
4. **Manage service configurations** and updates
5. **Handle errors** and implement best practices

## Prerequisites

- Python 3.7+
- `requests` library
- Access to a NDP EP API instance **version >= 0.1.0**
- Valid authentication credentials

> ⚠️ **API Version Requirement**: This tutorial requires NDP EP API version 0.1.0 or higher. The service management and redirect endpoints demonstrated in this tutorial are not available in earlier versions.

## API Overview

The NDP EP API provides service management endpoints that allow you to:
- Create services that proxy to external APIs
- Use redirect functionality to forward requests
- Manage service configurations and metadata
- Monitor service usage and performance

In [ ]:
# Install required packages
!pip install requests -q

## 1. Setup and Configuration

First, let's import the necessary libraries and configure our API connection parameters.

In [ ]:
import requests
import json
from typing import Dict, Any, Optional
import time
import datetime

# Pretty printing for JSON responses
from pprint import pprint

### Configuration Variables

**Important:** Replace these values with your actual API endpoint and credentials.

In [ ]:
# API Configuration
API_BASE_URL = "http://localhost:8003"  # Replace with your API URL
CKAN_SERVER = "local"  # Options: "local" or "pre_ckan"

# Authentication Token
# You can obtain this token from your Keycloak instance or use the /token endpoint
AUTH_TOKEN = "your_token"  # Replace with your actual token

# Request headers with authentication
HEADERS = {
    "Authorization": f"Bearer {AUTH_TOKEN}",
    "Content-Type": "application/json"
}

# External service configuration
DOGS_API_BASE_URL = "https://dog.ceo/api/breeds"

print(f"API Base URL: {API_BASE_URL}")
print(f"CKAN Server: {CKAN_SERVER}")
print(f"Dogs API URL: {DOGS_API_BASE_URL}")
print(f"Token configured: {'✓' if AUTH_TOKEN != 'your_token_from_keycloak' else '✗ Please set your token'}")

### Helper Functions

Let's create some utility functions to make our API interactions cleaner and more robust.

In [ ]:
def make_api_request(method: str, endpoint: str, data: Optional[Dict] = None, 
                    params: Optional[Dict] = None, use_auth: bool = True) -> Dict[str, Any]:
    """
    Make an API request with proper error handling.
    
    Args:
        method: HTTP method (GET, POST, PUT, PATCH, DELETE)
        endpoint: API endpoint (e.g., '/service', '/redirect')
        data: Request payload for POST/PUT/PATCH requests
        params: Query parameters
        use_auth: Whether to use authentication headers
    
    Returns:
        Dictionary containing the response data
    """
    url = f"{API_BASE_URL}{endpoint}"
    headers = HEADERS if use_auth else {"Content-Type": "application/json"}
    
    try:
        response = requests.request(
            method=method,
            url=url,
            headers=headers,
            json=data,
            params=params
        )
        
        print(f"🔗 {method} {url}")
        if params:
            print(f"📋 Params: {params}")
        print(f"📊 Status: {response.status_code}")
        
        if response.status_code in [200, 201]:
            try:
                result = response.json()
                print("✅ Success!")
                return result
            except json.JSONDecodeError:
                print("✅ Success! (No JSON response)")
                return {"success": True, "text": response.text}
        else:
            print(f"❌ Error: {response.status_code}")
            try:
                error_detail = response.json()
                print(f"Error details: {json.dumps(error_detail, indent=2)}")
                return {"error": True, "status_code": response.status_code, "detail": error_detail}
            except json.JSONDecodeError:
                print(f"Error text: {response.text}")
                return {"error": True, "status_code": response.status_code, "detail": response.text}
            
    except requests.exceptions.RequestException as e:
        print(f"❌ Request failed: {e}")
        return {"error": True, "exception": str(e)}

def print_response(response: Dict[str, Any], title: str = "Response"):
    """
    Pretty print API responses.
    """
    print(f"\n📋 {title}:")
    print("─" * 50)
    pprint(response)
    print("─" * 50)

def test_external_api(url: str) -> bool:
    """
    Test if an external API is accessible.
    """
    try:
        response = requests.get(url, timeout=10)
        return response.status_code == 200
    except:
        return False

## 2. API Status and External Service Test

Let's verify that both our NDP EP API and the external Dogs API are accessible.

In [ ]:
# Test NDP EP API connectivity
print("🔍 Testing NDP EP API connectivity...")
status_response = make_api_request("GET", "/status")
print_response(status_response, "NDP EP API Status")

if "error" not in status_response:
    print("🎉 NDP EP API is accessible and responding correctly!")
else:
    print("⚠️  NDP EP API connectivity issues. Please check your configuration.")

print("\n" + "="*60 + "\n")

# Test external Dogs API connectivity
print("🐕 Testing Dogs API connectivity...")
dogs_accessible = test_external_api(DOGS_API_BASE_URL)

if dogs_accessible:
    print(f"✅ Dogs API is accessible at {DOGS_API_BASE_URL}")
    
    # Get a sample of what the Dogs API returns
    try:
        sample_response = requests.get(DOGS_API_BASE_URL, timeout=10)
        if sample_response.status_code == 200:
            dogs_data = sample_response.json()
            print_response(dogs_data, "Dogs API Sample Response")
    except Exception as e:
        print(f"⚠️  Could not get sample data: {e}")
else:
    print(f"❌ Dogs API is not accessible at {DOGS_API_BASE_URL}")
    print("⚠️  This tutorial requires internet connectivity to access the Dogs API")

## 3. Service Creation

Now we'll create a new service called "dogs" that will proxy requests to the Dogs CEO API.

### Check Existing Services

First, let's see what services already exist in the system.

In [ ]:
# Check existing services using search endpoint
print("📋 Checking existing services...")

# Use the search endpoint to find services in the "services" organization
search_params = {
    "owner_org": "services",
    "server": "local"
}

services_response = make_api_request("POST", "/search", data=search_params)

if "error" not in services_response:
    print(f"\n📈 Found {len(services_response)} existing services")
    if services_response:
        print("Services:")
        for i, service in enumerate(services_response, 1):
            service_name = service.get('name', 'Unknown')
            # Get URL from the first resource if available
            service_url = 'No URL'
            if service.get('resources') and len(service['resources']) > 0:
                service_url = service['resources'][0].get('url', 'No URL')
            print(f"  {i}. {service_name} → {service_url}")
    else:
        print("  No existing services found")
else:
    print("⚠️  Unable to retrieve services list")
    if services_response.get('status_code') == 404:
        print("💡 The search endpoint may not be available in this API version")
    elif services_response.get('status_code') == 422:
        print("💡 Search parameters may be invalid for this API version")

### Create the Dogs Service

Now let's create our "dogs" service that will connect to the Dogs CEO API.

In [ ]:
# Generate timestamp for unique service name
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# Service configuration - following the ServiceRequest model
service_data = {
    "service_name": f"dogs_tutorial_{timestamp}",  # Unique name
    "service_title": "Dogs CEO API Service Tutorial",
    "owner_org": "services",  # Required to be exactly "services"
    "service_url": "https://dog.ceo/api",  # Base URL without /breeds
    "service_type": "external_api",
    "notes": "Service integration with the Dog CEO API (https://dog.ceo) for accessing dog breed information, images, and related data. This service demonstrates external API integration through the NDP EP platform.",
    "extras": {
        "provider": "Dog CEO",
        "api_version": "1.0",
        "documentation": "https://dog.ceo/dog-api/documentation/",
        "rate_limit": "No explicit rate limit",
        "authentication": "None required",
        "response_format": "JSON",
        "created_via": "NDP EP Tutorial",
        "creation_timestamp": timestamp,
        "purpose": "Educational tutorial and API integration demonstration"
    },
    "health_check_url": "https://dog.ceo/api/breeds/list",  # Optional
    "documentation_url": "https://dog.ceo/dog-api/documentation/"  # Optional
}

print("🐕 Creating Dogs service with configuration:")
print_response(service_data, "Service Configuration")

# Create the service using the correct endpoint
service_response = make_api_request("POST", "/services", data=service_data, params={"server": CKAN_SERVER})
print_response(service_response, "Service Creation Response")

if "error" not in service_response:
    print("\n🎉 Dogs service created successfully!")
    service_id = service_response.get('id') or service_data['service_name']
    print(f"🆔 Service ID: {service_id}")
    print(f"🌐 Service URL: {service_data['service_url']}")
    print(f"📝 Service Name: {service_data['service_name']}")
else:
    print("❌ Failed to create Dogs service")
    # Check if it's because service already exists
    if service_response.get('status_code') == 409:
        print("💡 Service may already exist. Trying to use existing service...")
        service_id = service_data['service_name']
    else:
        service_id = None

## 4. Testing Service Redirect Functionality

Now we'll test the redirect functionality by making requests through our NDP EP service to the Dogs API.

### Basic Redirect Test

Let's start with a basic redirect to the Dogs API base endpoint.

In [ ]:
if service_id:
    print("🔄 Testing basic service redirect...")
    
    # Use the service name as identifier for redirect
    service_name = service_data['service_name'] if 'service_data' in locals() else service_id
    print(f"📋 Using service identifier: {service_name}")
    
    # Test redirect to the base Dogs API endpoint
    # The correct endpoint is /services/redirect/{service_identifier}
    redirect_url = f"{API_BASE_URL}/services/redirect/{service_name}"
    
    print(f"🔗 Making direct request to: {redirect_url}")
    
    # Make direct request instead of using make_api_request to avoid routing issues
    try:
        response = requests.get(redirect_url, headers=HEADERS, timeout=30)
        print(f"📊 Status: {response.status_code}")
        
        if response.status_code == 200:
            print("✅ Success!")
        else:
            print(f"❌ Error: {response.status_code}")
    except requests.exceptions.RequestException as e:
        print(f"❌ Request failed: {e}")
else:
    print("⚠️  No service available for redirect testing")

### Advanced Redirect Test: List All Breeds

Now let's test the redirect functionality with the `/list/all` endpoint to get all dog breeds.

In [ ]:
if service_id:
    print("🐕 Testing redirect with /breeds/list/all endpoint...")
    
    # Use the service name as identifier
    service_name = service_data['service_name'] if 'service_data' in locals() else service_id
    
    # Test redirect to the list all breeds endpoint
    # The path will be appended to the service base URL
    path = "breeds/list/all"
    redirect_url = f"{API_BASE_URL}/services/redirect/{service_name}/{path}"
    
    print(f"📡 Making redirect request to: {service_data.get('service_url', 'Unknown')}/{path}")
    print(f"🔗 Using endpoint: {redirect_url}")
    
    # Make direct request to avoid routing to documentation endpoints
    try:
        response = requests.get(redirect_url, headers=HEADERS, timeout=30)
        print(f"📊 Status: {response.status_code}")
        
        if response.status_code == 200:
            try:
                result = response.json()
                print("✅ Success!")
                
                if "message" in result and isinstance(result["message"], dict):
                    print("\n🎉 /breeds/list/all redirect successful!")
                    breed_data = result["message"]
                    breed_count = len(breed_data)
                    
                    print(f"📊 Retrieved {breed_count} dog breeds")
                    print("\n🔤 First 10 breeds:")
                    
                    for i, (breed, sub_breeds) in enumerate(list(breed_data.items())[:10]):
                        sub_count = len(sub_breeds) if sub_breeds else 0
                        sub_info = f" ({sub_count} sub-breeds)" if sub_count > 0 else ""
                        print(f"  {i+1:2d}. {breed.capitalize()}{sub_info}")
                        
                        if sub_breeds and len(sub_breeds) <= 3:  # Show sub-breeds if few
                            for sub_breed in sub_breeds:
                                print(f"      → {sub_breed}")
                    
                    if breed_count > 10:
                        print(f"      ... and {breed_count - 10} more breeds")
                else:
                    print("❌ Unexpected response format")
                    print_response(result, "Response")
                    
            except json.JSONDecodeError:
                print("✅ Success! (Non-JSON response)")
                print(f"Response: {response.text[:200]}...")
        else:
            print(f"❌ Error: {response.status_code}")
            try:
                error_detail = response.json()
                print(f"Error details: {json.dumps(error_detail, indent=2)}")
            except:
                print(f"Error text: {response.text}")
    except requests.exceptions.RequestException as e:
        print(f"❌ Request failed: {e}")
else:
    print("⚠️  No service available for redirect testing")